# Amazon Bedrock Knowledge Bases가 지원하는 Query Reformulation

RAG 기반 GenAI 애플리케이션을 개발할 때 품질, 비용, 지연 시간을 최적화하는 것이 매우 중요합니다. Foundation Model(FM)에 전달되는 입력 질의는 종종 여러 질문과 복잡한 관계를 포함해 매우 복잡할 수 있습니다. 이렇게 복잡한 질의는 임베딩 단계에서 일부 핵심 요소가 희석되거나 가려져, 질의의 모든 측면을 포함하지 못하는 청크가 검색될 수 있습니다. 그 결과 RAG 애플리케이션이 기대 이하의 응답을 생성할 수도 있습니다.

Query reformulation을 사용하면 복잡한 입력 프롬프트를 여러 하위 질의로 분해할 수 있습니다. 각 하위 질의는 별도로 관련 청크를 검색하고, 검색된 청크를 통합한 뒤 순위를 매겨 FM에 전달해 응답을 생성합니다. 이 기능은 운영 환경에서 마주칠 수 있는 복잡한 질의에 대한 정확도를 높이는 데 사용할 수 있는 또 하나의 도구입니다.

# 노트북 설정
호환되는 역할과 컴퓨팅 환경을 준비한 뒤 아래 단계를 따라 진행하세요

In [ ]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
# Print sys.path to verify
# print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase

In [ ]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

In [ ]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"
knowledge_base_name_standard = 'standard-kb'
knowledge_base_description = "Octank 10k KB"
bucket_name = f'{knowledge_base_name_standard}-{suffix}'

foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

data_source=[{"type": "S3", "bucket_name": bucket_name}]

## 2 - 고정 청킹 전략으로 Knowledge Base 생성

In [ ]:
knowledge_base_standard = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name_standard}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source,
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{suffix}-f'
)

## 2.1 데이터 세트를 Amazon S3에 업로드
Knowledge Base를 만들었으니 `Octank financial 10K` 보고서 데이터 세트로 내용을 채워 보겠습니다. Knowledge Base 데이터 소스는 연결된 S3 버킷에 데이터가 있어야 하며, 데이터 변경 사항은 `StartIngestionJob` API 호출을 통해 Knowledge Base와 동기화할 수 있습니다. 이 예제에서는 헬퍼 클래스를 통해 API의 [boto3 추상화](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/start_ingestion_job.html)를 사용합니다. 

먼저 `dataset` 폴더에 있는 메뉴 데이터를 S3에 업로드합니다.

In [ ]:
import os

def upload_directory(path, bucket_name):
    for root, dirs, files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root, file)
            if file not in ["LICENSE", "NOTICE", "README.md"]:
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload, bucket_name, file)
            else:
                print(f"Skipping file {file_to_upload}")

upload_directory("../synthetic_dataset", bucket_name)


In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_standard.start_ingestion_job()

In [ ]:
kb_id = knowledge_base_standard.get_knowledge_base_id()

# Query Reformulation 실습

이 노트북에서는 query reformulation으로 이점을 얻을 수 있는 단순한 질의와 더 복잡한 질의를 살펴보고, 생성된 응답이 어떻게 달라지는지 확인합니다. 

## 복합 프롬프트

동작을 보여 주기 위해 Octank 10K 재무 문서에 포함된 정보를 여러 가지 요구사항으로 묻는 질의를 살펴보겠습니다. 이 질의에는 의미적으로 연관성이 낮은 여러 요청이 포함돼 있습니다. 이 질의를 검색 단계에서 임베딩하면 질의의 일부 의미가 희석되어, 복잡한 질의의 모든 요소를 다루지 못하는 청크가 반환될 수 있습니다.

Knowledge Base를 질의하고 응답을 생성하기 위해 __retrieve_and_generate__ API 호출을 사용합니다. Query reformulation 기능을 활성화하려면 아래와 같이 Knowledge Base 구성에 추가 정보를 포함합니다:

```
'orchestrationConfiguration': {
        'queryTransformationConfiguration': {
            'type': 'QUERY_DECOMPOSITION'
        }
    }
```

__참고:__ Query reformulation을 사용하지 않은 일반 __retrieve_and_generate__ 호출과 동일한 응답 구조를 반환합니다.

#### Query Reformulation 없이

Query reformulation을 사용하지 않고 다음 질의에 대한 생성 결과가 어떻게 보이는지 확인해 보겠습니다: 

"Where is the Octank company waterfront building located and how does the whistleblower scandal hurt the company and its image?"

In [ ]:
query = "What is octank tower and how does the whistleblower scandal hurt the company and its image?"

In [ ]:
response_ret = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)


# generated text output

print(response_ret['output']['text'],end='\n'*2)

In [ ]:
response_without_qr = response_ret['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_without_qr))
def citations_rag_print(response_ret):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret,1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

citations_rag_print(response_without_qr)

위 인용에서 알 수 있듯이, 복잡한 질의를 그대로 사용한 검색은 건물 관련 청크를 반환하지 못하고, 대신 whistleblower 사건과 가장 유사한 임베딩에 집중했습니다. 

이는 질의를 임베딩하는 과정에서 질의 일부 의미가 희석되었음을 시사합니다.

#### Query Reformulation 적용

이제 query reformulation이 더 정렬된 컨텍스트 검색에 어떤 도움을 주고, 그 결과 응답 정확도가 어떻게 향상되는지 살펴보겠습니다.

In [ ]:
response_ret = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            },
            'orchestrationConfiguration': {
                'queryTransformationConfiguration': {
                    'type': 'QUERY_DECOMPOSITION'
                }
            }
        }
    }
)


# generated text output

print(response_ret['output']['text'],end='\n'*2)

Query reformulation을 사용했을 때 검색된 청크를 살펴보겠습니다.

In [ ]:
response_with_qr = response_ret['citations'][0]['retrievedReferences']
print("# of citations or chunks used to generate the response: ", len(response_with_qr))


citations_rag_print(response_with_qr)

Query reformulation을 활성화하면 검색된 청크가 whistleblower 사건과 워터프런트 부동산 위치 모두에 대한 컨텍스트를 제공하는 것을 확인할 수 있습니다.

### CloudWatch 로그로 프롬프트 분해 관찰

검색을 수행하기 전에 복잡한 질의가 여러 하위 질의로 분해됩니다. 위 예제 질의의 분해 작업을 살펴보면, __standalone_question__에 원래 질의가 있고 결과 하위 질의는 __\<query\>__ 태그 사이에 표시됩니다.

__참고__: CloudWatch에서 로그를 확인하려면 Bedrock에서 호출 로깅을 활성화해야 합니다. 자세한 내용은 [여기](https://docs.aws.amazon.com/bedrock/latest/userguide/model-invocation-logging.html)를 참고하세요.

```
<generated_queries>

<standalone_question>
What is octank tower and how does the whistleblower scandal hurt the company and its image?
</standalone_question>

<query>
What is octank tower?
</query>

<query>
What is the whistleblower scandal involving Octank company?
</query>

<query>
How did the whistleblower scandal affect Octank company's reputation and public image?
</query>

</generated_queries>
```

<div class="alert alert-block alert-warning">
<b>참고:</b> 비용이 발생하지 않도록 KB, OSS 인덱스 및 관련 IAM 역할과 정책을 반드시 삭제하세요.
</div>

In [ ]:
print("===============================Knowledge base with fixed chunking==============================\n")
knowledge_base_standard.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)

Query reformulation이 작동하는 방식과 복잡한 질의의 응답을 어떻게 개선하는지 살펴본 만큼, 이 기법을 더 깊이 탐구하고 실험하여 RAG 워크플로를 최적화해 보세요. 